<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat</span>
<span><img src="media/inria-25-alpha.png" /></span>
</div>

# `git` tutorial (6)

## synchronisations entre repos

In [ ]:
# ce sera toujours notre façon de commencer
[ -f scripts/helpers.sh ] && source scripts/helpers.sh

## architecture décentralisée

* on a vu plusieurs architectures permettant de créer des workflows
* comme par exemple

![](media/archi-star.png)

* dans ce schéma 
  * les boites sont des repos
  * les liens correspondent à des synchronisations entre repos  

## notre exemple

pour cette partie :

* nous repartons d'un repo quasiment vide `initial-repo`
* que l'on va dupliquer (dans le jargon git: cloner) dans `cloned-repo`
* puis modifier, et synchroniser entre eux 

In [ ]:
cd $TOPLEVEL

# on recommence un autre repo plus simple; à nouveau
# je nettoie complètement ce qu'on a pu faire précédemment
if [ -d initial-repo ]; then
    echo "on repart d'un directory vide"
    rm -rf initial-repo
fi

# on le crée
mkdir initial-repo

# on va dedans
cd initial-repo

In [ ]:
# si nécessaire, on se place dans le repo git
[ -d initial-repo ] && cd initial-repo

pwd

## création

In [ ]:
$SCRIPTS/do populate-initial-repo

In [ ]:
git l

## le point

In [ ]:
# vous devez avoir 2 commits
git l


In [ ]:
# avec deux branches `master` et `devel`
# on est sur la branch devel
git branch

## pour les besoins du cours

* en général, les repos sont créés sur des **machines distinctes**
  * typiquement un repo local et un sur github
* techniquement, pas obligatoire
  * **pour les besoins du cours**, nous allons créer nos repos **localement** 
* les mécanismes de copie/synchronisation sont exactement identiques

In [ ]:
# on utilisera ce répertoire $TOPLEVEL/cloned-repo
# pour héberger un repo avatar de ce qu'on 
# pourrait mettre sur github

if [ -d $TOPLEVEL/cloned-repo ]; then
    rm -rf $TOPLEVEL/cloned-repo
fi

## `git clone` sert à dupliquer un repo

In [ ]:
cd $TOPLEVEL

git clone initial-repo/.git cloned-repo

In [ ]:
cd $TOPLEVEL/cloned-repo

ls

le clone va contenir:

* **les commits**, c'est-à-dire la partie `bare`
* les fichiers présents dans le **commit courant**
* mais par contre l'index n'est **pas concerné**
  * si on avait eu des modifications pendantes dans `initial-repo`
  * que ce soit dans l'index ou les fichiers
  * ils **n'auraient pas** été copiés

In [ ]:
git status

In [ ]:
git l

* en réalité dans un `git clone`
  * du coté source, seul le ***bare repo*** est lu
  * les fichiers et l'index ne sont **pas du tout regardés**
  * bien souvent d'ailleurs la source est sur une infra comme github
  * et dans ce cas la source n'est **que** un *bare repo*
  
* les fonctions de synchro entre repos
  * ne concernent en réalité  
    que la partie 'bare repo' des deux cotés
  * en général les fichiers et index **ne sont pas concernés**  
    par les synchros entre repo
  * qui ne font principalement que transférer des commits  
    (et mettre à jour des branches)
    
* sauf dans le cas de `pull` 
  * qui en réalité fait `fetch` + `merge`
  * et du coup `merge` peut être amené à toucher les fichiers

à noter surtout:

* **pas de hiérarchie** entre les repos
  * la source et le clone sont des **pairs** - pas de *master*/*slave* 
* la seule différence bien sûr ce sont les droits d'accès
  * en gros c'est **chacun chez soi**
  * si vous avez les droits d'accès (linux, windows, macOS)  
    sur un repo vous pouvez écrire dedans

## `git clone` en vrai 

l'usage le plus fréquent consiste à dupliquer un repo qui est publié sur `github`

par exemple ce cours est sur
https://github.com/flotpython/gittutorial/

![](media/github-clone.png)

tapez 

    git clone 
    
et faites 'coller' avec Control-V

    git clone git@github.com:flotpython/gittutorial.git
    
vous pouvez alors faire

    cd gittutorial
    git log --oneline
    ls

## les fonctions de synchronisation

en plus de `clone`, les fonctions de synchronisation sont:

* `fetch`
* `pull`
* `push`

**passif**
* `fetch`  injecte des commits distants dans le repo local **sans impact local**

**actif**
* `push`: injecte des commits locaux dans le repo distant
* `pull` = `fetch`+`merge`


## la notion de *remote*

avant de voir en détail les fonctions de synchro,  
nous devons voir la notion de *remote*

un *remote*, c'est 
* essentiellement **un nom** symbolique
* qui nous permet de faire facilement référence à un **autre repo**
* i.e. plutôt que de retaper **son URL** à chaque fois

## les *remote*s

dans notre clone, notez la présence d'un *remote* appelé `origin`

In [ ]:
# nous sommes dans le clone
pwd

In [ ]:
# pour lister les remotes connus
git remote

au moment du `clone`, git a créé pour nous ce *remote* avec le nom prédéfini `origin`, qui désigne le repo d'où on a cloné

In [ ]:
# en version bavarde on voit à quoi correspond le remote 
git remote -v

comme pour les branches, on peut facilement ajouter, renommer, etc. les *remote*s

## branches et remotes

un repo git est *self-contained*

* toutes les références (branche et remote) sont des **objets locaux**
* on peut toujours travailler sans connexion réseau

pour résumer, deux notions très différentes

* la branche désigne un point dans les commits (forcément locaux)
* le remote est simplement une **référence** vers un autre repo
  * c'est juste un nom, un alias, vers un autre repo
* ainsi par exemple
  * on peut sans souci créer un remote vers un repo inexistant
  * c'est seulement quand on s'en sert - via fetch/push/pull - qu'on se rendra compte du problème

## branches et remotes - suite

* comme un repo est *self-contained*
* il conserve **localement** la trace des branches distantes
* voyez par exemple les branches `origin/master` et `origin/devel`


In [ ]:
git l -3 --all

qu'on pourrait rephraser comme:
* du point de vue du repo `cloned-repo`
* il y a dans le repo distant `origin` (donc, `my-first-repo`)
* une branche `master` 
* qui pointe vers ce commit

In [ ]:
# REMARQUE
# on peut utiliser la syntaxe origin/devel 
# pour désigner un commit
git l origin/devel

## branches et remotes - suite

cette information **n'est pas** garantie d'être 100% à jour !

on va le voir tout de suite : 

* si je crée dans `my-first-repo` un commit
  * c'est une opération **strictement locale**
* le clone `cloned-repo` n'en n'est pas informé immédiatement
  * à nouveau, c'est du pair à pair / chacun chez soi
* il le sera essentiellement s'il fait un `fetch`

remarque:

* on peut configurer énormément de choses; par exemple
  * décider de synchoniser deux repos après chaque commit
* mais c'est de l'ordre du confort
  * ce sont des **opérations élémentaires distinctes**

## synchro - fetch

notre scénario

* créer u nouveau commit dans le repo originel `my-first-repo`
* observer les deux repos à ce stade
* déclencher un `fetch` depuis `cloned-repo`
* observer les deux repos à ce stade

## fetch (1) - créer un commit à l'origine

In [ ]:
cd $TOPLEVEL/my-first-repo

In [ ]:
git l


In [ ]:
$SCRIPTS/do no-worries-2
git add factorial.py
git commit -m "fix factorial in the origin repo"

In [ ]:
git l

## fetch (2) - utiliser `fetch` depuis le clone

In [ ]:
# je retourne sur le clone
cd $TOPLEVEL/cloned-repo

In [ ]:
git l

In [ ]:
# pour aller chercher 
git fetch --all

## après le `fetch`

In [ ]:
git l --all

remarquez :

* nous voyons un nouveau commit !
* `origin/master` est mis à jour
* ainsi d'ailleurs que `origin/HEAD`
* remarque: pas de branch locale `devel`
  * on peut cependant faire
  * `git checkout devel` 

## mettre à jour les références locales

à ce stade, pour me mettre à jour par rapport au repo distant, je peux

* merger `origin/master` dans `master`
* ce qui ferait avancer `master` d'un cran 
* noter que c'est un merge `fast-forward`
* et donc, pas de création de commit

In [ ]:
git merge origin/master

In [ ]:
git l

## `pull = fetch + merge`

* c'est exactement le propos de `pull`
* que d'automatiser ce genre d'opérations
* en une seule passe

plusieurs formes

* `git pull origin master` 
  * met à jour `origin/master` depuis le remote `origin`
  * et le merge dans `master`
* `git pull origin master:devel` 
  * met à jour `origin/master` et le merge dans `devel`

## `pull` et raccourcis

* on peut même encore raccourcir, grâce à la configuration
* en partant du nom de la branche courante, ici `master`
* qui a été configurée lorsqu'on a fait `clone`

In [ ]:
# git config permet de lire
# un attribut dans la config
git config branch.master.remote

In [ ]:
git config branch.master.merge

du coup je peux encore enlever des arguments

* `git pull origin`
  * revient à `git pull origin master`
* `git pull` 
  * revient `git pull origin master`

## résumé sur *fetch* et *merge* et *pull*

pour résumer jusqu'ici :

* `fetch`
  * avec `git fetch` on sait aller chercher 
    les commits présents dans les autres repos
  * je recommande de faire `git fetch --all` 
    car la commande est principalement inoffensive
  * les outils genre SourceTree font cela périodiquement par défaut

* `merge` 
  * on peut ensuite merger ces commits
  * exactement comme si on les avait créés localement
  * notamment vis-à-vis des *fast-forward* 
  
* `pull` 
  * permet de faire les deux phases en une fois

## dans l'autre sens


* le modèle étant symétrique (pair à pair)
* à première vue, on se dit que le *push* 
* c-à-d propager des commits locaux vers un repo ditant
* devrait être l'exact symétrique du *pull*


en pratique ce n'est **pas le cas**

## dissymétrie

la dissymétrie est liée à la résolution de conflits :

* lors d'un `pull`, il y a un humain qui peut résoudre les conflits, revenir en arrière, etc..
* lors d'un `push`, ce **n'est pas forcément le cas**

c'est pourquoi :

* l'opération de `push` est effectivement l'inverse de `pull`
* on recopie à distance les commits qui n'y sont pas encore
* et on merge dans la branche distante
* mais c'est **limité à des *fast-foward***

## push et droits d'accès

notez aussi que, bien entendu, lors d'un push :
* il faut les droits d'écriture dans le repo distant

dans mon cas de figure
* puisque mes deux repos sont locaux
* j'ai bien le droit d'écrire dans les deux repos 

dans le cas d'un repo distant sur github (gitlab, ...)
* il faut faire une démarche particulière pour obtenir ce droit
* **ou bien** se créer un *fork* (c'est leur principale raison d'être)
* on reparlera de tout ça

## un push simple

scénario #1 : un push qui se passe bien

* je crée un commit dans le repo original
* je le pousse sur le clone

## 

## un push compliqué

scénario #2 : un push qui ne se passe pas bien

* en partant des deux repo synchronisés
* je crée un commit sur l'un et un comnit sur l'autre
* si j'essaie de pousser à ce stade
  * je ne suis pas dans le cas *fast forward* !
  * et le *push* échoue*
* il me faut alors d'abord tirer, avant de pouvoir pousser